In [0]:
%python
dbutils.library.installPyPI('azure-cosmos', version='3.1.2')

Out[1]: True

In [0]:
%fs ls /mnt/

path,name,size
dbfs:/mnt/CSI.010,CSI.010,17819
dbfs:/mnt/CSIACK.010,CSIACK.010,16193
dbfs:/mnt/INSTest.030,INSTest.030,515
dbfs:/mnt/dataanalysis/,dataanalysis/,0
dbfs:/mnt/ediparser-package-source/,ediparser-package-source/,0
dbfs:/mnt/ipi-integration/,ipi-integration/,0
dbfs:/mnt/postgresql-archive/,postgresql-archive/,0
dbfs:/mnt/reporting/,reporting/,0
dbfs:/mnt/result.json,result.json,435
dbfs:/mnt/royalty/,royalty/,0


In [0]:
from datetime import datetime, timedelta, date

dbutils.widgets.removeAll()

frmDate= date.today() - timedelta(days=7)
dbutils.widgets.text("Agency Code (zero filled)", "All")
dbutils.widgets.text("From Date",frmDate.strftime("%d/%m/%Y"))
dbutils.widgets.text("To Date",date.today().strftime("%d/%m/%Y"))

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pyspark.sql.functions as func


def __parquet_file_name(date):
  container = 'mnt/submission-audit/'
  return datetime.strftime(date, container + '%Y/%m/%d.parquet')

def __get_date_range(from_date,to_date):
  delta = to_date - from_date
  print(delta.days)
  return [from_date + timedelta(x) for x in range(delta.days)]

def file_exists(path: str):
  try:
      dbutils.fs.ls(path)
      return path
  except Exception as e:
      if 'java.io.FileNotFoundException' in str(e):
          return None
      else:
          raise

# Set the date range from the widget
fromDate = datetime.strptime(dbutils.widgets.get("From Date"), '%d/%m/%Y')
toDate=datetime.strptime(dbutils.widgets.get("To Date"), '%d/%m/%Y')
filedates = __get_date_range(fromDate,toDate)

# Read all the files in the data range
parquet_files = list(
  filter(None, map(file_exists, map(__parquet_file_name, filedates))))

data = spark.read.parquet(*parquet_files)


# Shape data by creating a date only column and setting the Code to 000 where it is null
# Code 000 means the transaction was succesful
# Also create an hour bucket column so that we can group transactiosn by hour
data=data.withColumn('CreatedDateOnly',col('CreatedDate').cast('date'))
data=data.withColumn('HourBucket',func.hour(col('CreatedDate')))
data=data.withColumn('DbCode', func.lpad(col('SourceDb'),3, '0'))
data=data.fillna({'Code':'000'})

display(data)

AuditRequestId,AuditId,RecordId,AgencyCode,CreatedDate,IsProcessingError,IsProcessingFinished,Code,Message,TransactionType,PreferredIswc,AgencyWorkCode,SourceDb,OriginalTitle,TransactionSource,CreatorNames,CreatorNameNumbers,PublisherNameNumber,PublisherWorkNumber,RelatedSubmissionIncludedIswc,AgentVersion,CreatedDateOnly,HourBucket,DbCode
b1faf98a-4de7-4ccb-944a-f37bb07ecb14,1f5160d8-36b5-4edb-a8fe-fc263e989bc3,1,,2021-09-08T10:00:32.221+0000,true,true,103,Source Society Code is invalid,CAR,null,T000007273,128,UI TEST 359b23bdff2c 7f4ef1434115 daf82dd15a26,Agency,,,0,,false,,2021-09-08,10,128
9e921e34-c475-41d3-8225-e438282f3c89,1f5160d8-36b5-4edb-a8fe-fc263e989bc3,3,128,2021-09-08T10:00:32.221+0000,true,true,115,Specified source db is not allowed to make a submission for the specified society,CAR,null,000-007-286V,88,CWR SMOKETEST Y2N7CSKQOANJKSETQHP5 6F2WXULFY7KBWSIMCOY8BYGIB,Agency,,,0,,false,,2021-09-08,10,088
04f139de-f7a6-4b01-88f4-bde215213de2,9cd9c99f-146d-466e-a86e-872c8a8aeff2,0,021,2021-09-08T10:45:35.490+0000,false,true,000,null,CAR,T9804296637,0789437fighrejoca,21,6766 CUR ISSUE TEST WORK,Agency,GARLAND ORLA,854580022,0,,false,,2021-09-08,10,021
155c1224-7a7e-4d0c-9611-a848c5c2b079,97d77849-d53b-4274-8161-14da5def0f8d,0,021,2021-09-08T10:50:01.223+0000,false,true,000,null,CUR,T9804296637,0789437fighrejoca,21,6766 CUR ISSUE TEST WORK,Agency,,854580022,0,,true,,2021-09-08,10,021
0bb3370d-c8cb-4630-99cb-c7b8bf6e0268,dcd8ff12-d033-4190-81b0-8122b202bffa,0,021,2021-09-08T10:54:38.452+0000,false,true,000,null,CAR,T9804296648,90834fhoierva,21,6766 TEST WORK CUR INELIGIBLE ISSUE,Agency,LENNON JOHN WINSTON,17798450,0,,false,,2021-09-08,10,021
cbc16da7-12e4-46da-8121-b763520f4135,e0562596-7444-4337-83f1-0b82b3520ffd,0,021,2021-09-08T10:56:10.805+0000,false,true,000,null,CUR,T9804296648,90834fhoierva,21,6766 TEST WORK CUR INELIGIBLE ISSUE,Agency,,17798450,0,,true,,2021-09-08,10,021
da3a3514-68b6-49e8-8394-6cdcf14bfab6,1f5160d8-36b5-4edb-a8fe-fc263e989bc3,0,,2021-09-08T10:00:32.221+0000,true,true,103,Source Society Code is invalid,CAR,null,T000007271,128,UI TEST 566c81c7c88e a3a27cd031b7 adffda7d2739,Agency,,,0,,false,,2021-09-08,10,128
7a83bd22-05bb-45eb-b2e0-2a6ee92a2ed7,1f5160d8-36b5-4edb-a8fe-fc263e989bc3,2,,2021-09-08T10:00:32.221+0000,true,true,103,Source Society Code is invalid,CAR,null,T000007276,128,UI TEST c05fc8e6d5f6 3f4e5e171951 07dc60d1418f,Agency,,,0,,false,,2021-09-08,10,128
f59e19bf-c452-44e8-9134-4a82a00c8829,174d41a0-5596-4a21-a036-a8e3335985ce,2,,2021-09-08T11:00:06.407+0000,true,true,103,Source Society Code is invalid,CAR,null,T000007295,128,UI TEST 9ace0814bb9a 1f97dc428106 7f6952109433,Agency,,,0,,false,,2021-09-08,11,128
71c385f6-84b8-4f1d-91ad-b62e3510cd61,ba7f94f9-0e8a-4ae2-ac3e-0427da3b8582,0,003,2021-09-08T11:01:32.063+0000,true,true,108,The IP on the work submission is not a creator affiliated with the source society,CAR,null,bOwTd8dzP7A4as,300,INT TEST xjKbyuLtoR2642021625Ftas H296rhpRGw24170940406TWmJ,Agency,PUBLIC DOMAIN,865900903,0,,false,,2021-09-08,11,300


In [0]:
agencyfile=dbutils.fs.ls("/mnt/submission-audit/MasterData/Agencies.txt")
agencyfiles=[agencyfile[0].path]
agencydf=spark.read.csv(agencyfiles,header='true')
#display(agencydf)

agencydf = agencydf.withColumnRenamed('CreatedDate', 'AgencyCreatedDate')
dbdf = agencydf.select(col('AgencyID').alias('DbID'),col('Name').alias('DB Name'))

## Set DB Code for WID submissions societies submitting through Agency Portal
data = data.withColumn('DbCode', func.when(data['AgencyCode'].isin(['000','009','040','079','127','077','110','111','122','089','090','097']),'300').otherwise(data['DbCode']))

# Set db for Latinnet societies 
#Latinnet: 004(AGADU), 007( APDAYC), 015( APA ), 060 ( SACVEN ), 061(SADAYC ), 065( SAYCE ), 084 ( SAYCO ), 103 ( ACDAM  ), 107 (ACAM ), 129 (SOBODAYCOM  ), 146   SPAC(PANAMA) , 226  (AACIMH  ), 227   ( SGACEDOM   ), 241 ( NICAUTOR #  ), 242 ( SACIM  ), 250 ( AEI )
data = data.withColumn('DbCode', func.when(data['AgencyCode'].isin(['004','007','015','060','061','065','084','103','107','129','146','226','227','241','242','250']),'309').otherwise(data['DbCode']))

#
# Set db to 310 DIVA including resetting those through the WUD
# MCSC(119) and CASH(026) WAMI 269, MACP 104, MACA 265, MUST 161
data = data.withColumn('DbCode', func.when(data['AgencyCode'].isin(['119','026','104','265','161']),'310').otherwise(data['DbCode']))

data=data.join(agencydf,on=data.AgencyCode==agencydf.AgencyID,how="inner")
data=data.join(dbdf,on=data.DbCode==dbdf.DbID)
display(data.filter(col('DbCode')=='104'))

AuditRequestId,AuditId,RecordId,AgencyCode,CreatedDate,IsProcessingError,IsProcessingFinished,Code,Message,TransactionType,PreferredIswc,AgencyWorkCode,SourceDb,OriginalTitle,TransactionSource,CreatorNames,CreatorNameNumbers,PublisherNameNumber,PublisherWorkNumber,RelatedSubmissionIncludedIswc,AgentVersion,CreatedDateOnly,HourBucket,DbCode,AgencyID,Name,Country,AgencyCreatedDate,LastModifiedDate,LastModifiedUserID,ISWCStatus,DbID,DB Name


In [0]:
filterAgency= dbutils.widgets.get("Agency Code (zero filled)")
if(filterAgency != 'All'):
  data=data.filter(col('AgencyCode') == filterAgency)

In [0]:
# This report does exclude transactions that were technical rejections

data_summary_ok = data.select('DbCode','Db Name','CreatedDateOnly').filter(col('Code')!='100').groupBy('DbCode','Db Name','CreatedDateOnly').agg(func.count(func.lit(1)).alias("TransactionCount"))
display(data_summary_ok.sort('CreatedDateOnly','DbCode','Db Name'))



DbCode,Db Name,CreatedDateOnly,TransactionCount
000,DP,2021-09-06,1
003,AEPI,2021-09-06,1
010,ASCAP,2021-09-06,1
058,SACEM,2021-09-06,5
128,IMRO,2021-09-06,18
300,WIDCENTRE,2021-09-06,8
315,CSI,2021-09-06,1
000,DP,2021-09-07,12
003,AEPI,2021-09-07,4
010,ASCAP,2021-09-07,153


In [0]:
data_summary_ok_wid = data.select('DbCode','Db Name','AgencyCode','Name','CreatedDateOnly').filter(col('Code')!='100').filter(data['DbCode'].isin(['300','309','310'])) .groupBy('DbCode','Db Name','AgencyCode','Name','CreatedDateOnly').agg(func.count(func.lit(1)).alias("TransactionCount"))


display(data_summary_ok_wid.sort('DbCode','Db Name','CreatedDateOnly','AgencyCode','Name'))

DbCode,Db Name,AgencyCode,Name,CreatedDateOnly,TransactionCount
300,WIDCENTRE,003,AEPI,2021-09-06,8
300,WIDCENTRE,003,AEPI,2021-09-07,131
300,WIDCENTRE,038,JASRAC,2021-09-07,2
300,WIDCENTRE,052,PRS,2021-09-07,4
300,WIDCENTRE,122,AKKA-LAA,2021-09-07,94
300,WIDCENTRE,003,AEPI,2021-09-08,74
300,WIDCENTRE,038,JASRAC,2021-09-08,2
300,WIDCENTRE,052,PRS,2021-09-08,4
300,WIDCENTRE,122,AKKA-LAA,2021-09-08,94
300,WIDCENTRE,003,AEPI,2021-09-09,37


In [0]:
display(data_summary_ok.sort('CreatedDateOnly','DbCode','Db Name'))

DbCode,Db Name,CreatedDateOnly,TransactionCount
000,DP,2021-09-06,1
003,AEPI,2021-09-06,1
010,ASCAP,2021-09-06,1
058,SACEM,2021-09-06,5
128,IMRO,2021-09-06,18
300,WIDCENTRE,2021-09-06,8
315,CSI,2021-09-06,1
000,DP,2021-09-07,12
003,AEPI,2021-09-07,4
010,ASCAP,2021-09-07,153


In [0]:
# This report does exclude transactions that were technical rejections
dfb = [{}]
data_sum_byhour = data.select('CreatedDateOnly','HourBucket','AgencyCode').filter(col('Code')!='100').groupBy('CreatedDateOnly','HourBucket','AgencyCode').agg(func.count(func.lit(1)).alias("TransactionCount"))


display(data_sum_byhour.sort('CreatedDateOnly','HourBucket'))


CreatedDateOnly,HourBucket,AgencyCode,TransactionCount
2021-09-06,8,128,14
2021-09-06,16,058,4
2021-09-06,16,003,10
2021-09-06,16,128,5
2021-09-06,16,010,1
2021-09-06,16,123,1
2021-09-07,13,038,1
2021-09-07,13,128,106
2021-09-07,13,124,2
2021-09-07,13,021,203


In [0]:
# This report does exclude transactions that were technical rejections

display(data_summary_ok.sort('CreatedDateOnly','DbCode','Db Name'))


DbCode,Db Name,CreatedDateOnly,TransactionCount
315,CSI,2021-08-03,210
000,DP,2021-08-04,6
003,AEPI,2021-08-04,2
010,ASCAP,2021-08-04,80
021,BMI,2021-08-04,213
052,PRS,2021-08-04,18
058,SACEM,2021-08-04,114
124,COSOMA,2021-08-04,1
128,IMRO,2021-08-04,167
300,WIDCENTRE,2021-08-04,73


In [0]:
# This report does exclude transactions that were technical rejections (Code==100).  Sucesful transactions have Code == '000'
prs_summary_ok = data.select('CreatedDateOnly','Code','Message').filter((col('Code')!='100') & (col('Code')!= '000')).groupBy('CreatedDateOnly','Code','Message').agg(func.count(func.lit(1)).alias("TransactionCount"))
display(prs_summary_ok.sort('CreatedDateOnly','Code','Message'))

CreatedDateOnly,Code,Message,TransactionCount
2021-09-06,102,I.P. Name Number is required,1
2021-09-06,108,The IP on the work submission is not a creator affiliated with the source society,1
2021-09-06,110,Only one Original Title allowed,1
2021-09-06,111,At least one creator I.P. is required,3
2021-09-06,113,ISWC format error [T][0-9]{10},1
2021-09-06,115,Specified source db is not allowed to make a submission for the specified society,1
2021-09-06,119,The ISWC provided in the Derived From DF record is not a valid Preferred ISWC in the ISWC Database,1
2021-09-06,120,Each Derived From DF record must contain either an ISWC or a Title,2
2021-09-06,123,Invalid disambiguation reason code,1
2021-09-06,126,Performer information must contain a Second Name or a Second Name and a First Name,2


In [0]:
# Summary of all submissions technically rejected in the range by day and agency code

data_summary_rejected = data.select('DbCode','Db Name','CreatedDateOnly','AgencyWorkCode').filter(col('Code')=='100').groupBy('CreatedDateOnly','DbCode','Db Name','AgencyWorkCode').agg(func.countDistinct('AgencyWorkCode').alias("TransactionCount"))
display(data_summary_rejected.sort("CreatedDateOnly",'DbCode','Db Name'))

CreatedDateOnly,DbCode,Db Name,AgencyWorkCode,TransactionCount
2021-09-06,010,ASCAP,W4tFk0nVSRx,1
2021-09-06,010,ASCAP,psZ0fHnyArabYTSiK,1
2021-09-06,010,ASCAP,b2dDsJkqOgxrUN4,1
2021-09-06,010,ASCAP,iEe6RaNqX3,1
2021-09-06,010,ASCAP,c68mBz4JCFX3,1
2021-09-06,010,ASCAP,KdzWQlTdqaNzYU,1
2021-09-06,010,ASCAP,oStw7Sx1TcxT,1
2021-09-06,010,ASCAP,CbPaAyJJJA,1
2021-09-06,010,ASCAP,KUYeXFMsFwBhBJCmld,1
2021-09-06,010,ASCAP,9SiNO6lM0yQP,1


In [0]:
display(data.select('AgencyCode','Name','CreatedDateOnly','AgencyWorkCode').filter(col('Code')=='100'))

AgencyCode,Name,CreatedDateOnly,AgencyWorkCode
128,IMRO,2021-08-06,ssdAiYBwEfUug92DVy
128,IMRO,2021-08-06,j13UZyyoqJ3
128,IMRO,2021-08-06,dwa2liJM71oMcVqr8
128,IMRO,2021-08-06,uFPumGY2th
021,BMI,2021-08-06,naM6N0zA
128,IMRO,2021-08-05,i4FUf7o6KHecrOMX
128,IMRO,2021-08-05,i4FUf7o6KHecrOMX
128,IMRO,2021-08-05,6XXhxlfPD2Bx5J7R
128,IMRO,2021-08-05,YTLbvzCmo0JIWD
128,IMRO,2021-08-05,R57NT0JvNd


In [0]:
# List all Technical Failures
display( data.filter(col('Code')=='100'))

AuditRequestId,AuditId,RecordId,AgencyCode,CreatedDate,IsProcessingError,IsProcessingFinished,Code,Message,TransactionType,PreferredIswc,AgencyWorkCode,SourceDb,OriginalTitle,TransactionSource,CreatorNames,CreatorNameNumbers,PublisherNameNumber,PublisherWorkNumber,RelatedSubmissionIncludedIswc,AgentVersion,CreatedDateOnly,HourBucket,DbCode,AgencyID,Name,Country,AgencyCreatedDate,LastModifiedDate,LastModifiedUserID,ISWCStatus,DbID,DB Name
68517e9c-9ea9-4bed-94a7-24aaecbb8aa6,6443e9e4-9ce8-45b6-b02e-d51b18d604aa,0,128,2021-08-06T09:24:06.861+0000,true,true,100,Internal Server Error.,CUR,T9804124876,ssdAiYBwEfUug92DVy,128,INT TEST GD7sMsKpQX1519755376Tj8uE Hls5dFUF5g35411984360WwuK,Agency,BAKER BILL,1856409,0,,true,,2021-08-06,9,128,128,IMRO,IRELAND,2008-03-04 10:27:40.0000000,2008-03-04 10:27:40.0000000,7,1,128,IMRO
b3db0091-db90-4f02-9df9-0dac30455623,9b4e2f5d-6c68-4cad-8889-84047828460d,0,128,2021-08-06T10:00:00.313+0000,true,true,100,Internal Server Error.,CUR,T9804125528,j13UZyyoqJ3,128,INT TEST pOSOoiZwnI3105457542DKlHp cS5MsrmNbv2987017637yZBbV,Agency,BAKER BILL,1856409,0,,true,,2021-08-06,10,128,128,IMRO,IRELAND,2008-03-04 10:27:40.0000000,2008-03-04 10:27:40.0000000,7,1,128,IMRO
002f9a52-c3de-4ea8-8371-f38c3482f1df,02b4a82e-f657-4546-8e2b-5ce0698d99d3,0,128,2021-08-06T10:09:34.936+0000,true,true,100,Internal Server Error.,CUR,T9804125744,dwa2liJM71oMcVqr8,128,INT TEST S2Z7fMBWdm3143441335yISEI lTIMW52ZYU3362375947D9cjV,Agency,BAKER BILL,1856409,0,,true,,2021-08-06,10,128,128,IMRO,IRELAND,2008-03-04 10:27:40.0000000,2008-03-04 10:27:40.0000000,7,1,128,IMRO
522853e6-3a2a-4f7a-899d-654829a690ed,6e119d6e-e321-4c99-822c-055f2779678b,0,128,2021-08-06T10:01:01.712+0000,true,true,100,Internal Server Error.,CUR,T9804125631,uFPumGY2th,128,INT TEST yC2MJpFrpG240994193FjCYl sJyclaQALT1755180655VkLyk,Agency,BAKER BILL,1856409,0,,true,,2021-08-06,10,128,128,IMRO,IRELAND,2008-03-04 10:27:40.0000000,2008-03-04 10:27:40.0000000,7,1,128,IMRO
77b36d54-1026-4133-a607-01fcb7b73824,f6abac81-65ba-4e91-94f0-31e7476a1f20,0,021,2021-08-06T14:05:16.367+0000,true,true,100,Internal Server Error.,CUR,T9804138929,naM6N0zA,21,INT TEST tnbNPS0QQT892343077wkbYt FTDNbzVklm3254200103jxsLD,Agency,,,0,,true,,2021-08-06,14,021,021,BMI,UNITED STATES,2008-03-04 10:27:40.0000000,2008-03-04 10:27:40.0000000,7,1,021,BMI
f0fa1a91-7f6d-45f9-b87e-6d9ba673cb08,63a48238-a551-4379-9412-f595659b56ef,0,128,2021-08-05T12:38:30.267+0000,true,true,100,Internal Server Error.,CUR,T9804107173,i4FUf7o6KHecrOMX,128,INT TEST oNzTFLDnYK2049304150Zz5hR joAkN5GGoC26922357273GZ5g,Agency,BAKER BILL,1856409,0,,true,,2021-08-05,12,128,128,IMRO,IRELAND,2008-03-04 10:27:40.0000000,2008-03-04 10:27:40.0000000,7,1,128,IMRO
0d56334b-6f77-406e-9ea1-56192c054e73,3facd6b3-c228-463c-8df1-635b91848235,0,128,2021-08-05T12:39:39.931+0000,true,true,100,Internal Server Error.,CUR,T9804107173,i4FUf7o6KHecrOMX,128,INT TEST oNzTFLDnYK2049304150Zz5hR joAkN5GGoC26922357273GZ5g,Agency,BAKER BILL,1856409,0,,true,,2021-08-05,12,128,128,IMRO,IRELAND,2008-03-04 10:27:40.0000000,2008-03-04 10:27:40.0000000,7,1,128,IMRO
775d19c1-cc49-47fc-890c-b7ad72171df9,cf3a4773-80fe-4930-a092-9bdd132c594c,0,128,2021-08-05T13:27:20.227+0000,true,true,100,Internal Server Error.,CUR,T9804107264,6XXhxlfPD2Bx5J7R,128,INT TEST cv9oAEWUxH2033016231vZBlB tHp9WuayOq4258427020nCW0T,Agency,BAKER BILL,1856409,0,,true,,2021-08-05,13,128,128,IMRO,IRELAND,2008-03-04 10:27:40.0000000,2008-03-04 10:27:40.0000000,7,1,128,IMRO
cda63a6d-c91d-48bc-96a7-77282d2d834d,1ddc5ebe-92da-4128-a5b2-6ef921dba3cd,0,128,2021-08-05T13:09:02.003+0000,true,true,100,Internal Server Error.,CUR,T9804107208,YTLbvzCmo0JIWD,128,INT TEST d0LXU3VusV3408078847aH32d uZECDOaCUj314544431699WxY,Agency,BAKER BILL,1856409,0,,true,,2021-08-05,13,128,128,IMRO,IRELAND,2008-03-04 10:27:40.0000000,2008-03-04 10:27:40.0000000,7,1,128,IMRO
50c56cdb-680b-47df-9a80-ee7ab2c101c6,a9d68486-8278-4596-9c63-0f955d2691c8,0,128,2021-08-05T13:34:21.362+0000,true,true,100,Internal Server Error.,

In [0]:
display( data.filter(col('Code')=='127'))

AuditRequestId,AuditId,RecordId,AgencyCode,CreatedDate,IsProcessingError,IsProcessingFinished,Code,Message,TransactionType,PreferredIswc,AgencyWorkCode,SourceDb,OriginalTitle,TransactionSource,CreatorNames,CreatorNameNumbers,PublisherNameNumber,PublisherWorkNumber,RelatedSubmissionIncludedIswc,AgentVersion,CreatedDateOnly,HourBucket,DbCode,AgencyID,Name,Country,AgencyCreatedDate,LastModifiedDate,LastModifiedUserID,ISWCStatus,DbID,DB Name
2c9985b1-c192-45a7-bed3-fe467b549243,6c041638-9cb3-47a3-ba49-54572f9ece2b,0,122,2021-08-06T09:14:55.325+0000,true,true,127,Submitted titles don’t match current ISWC Documentation,CAR,T9804123055,5LTpv6HdHTgdDFrif,308,INT TEST zsyhrXgWhxgSvQf SbMdZaTqplJPOHv KNzVwQgLLHjFOvv x,Agency,BAKER BILL ;REEVES KEN,1856409;1906032,0,,false,,2021-08-06,9,300,122,AKKA-LAA,LATVIA,2008-03-04 10:27:40.0000000,2008-03-04 10:27:40.0000000,7,1,300,WIDCENTRE
dc60d984-e7e1-4c9b-a119-8505ad0086f0,77b8b7ed-bfb4-4af2-8587-f2453679fd2b,0,122,2021-08-06T10:33:32.607+0000,true,true,127,Submitted titles don’t match current ISWC Documentation,CAR,T9804127455,eZf3CVBN9rDT3,308,INT TEST ktoYOvqauLtDaXL hWgQKQPsXDSpufJ ezdxQIPURZjCvNa x,Agency,BAKER BILL ;REEVES KEN,1856409;1906032,0,,false,,2021-08-06,10,300,122,AKKA-LAA,LATVIA,2008-03-04 10:27:40.0000000,2008-03-04 10:27:40.0000000,7,1,300,WIDCENTRE
f7bb9819-1fff-4d05-b891-695ece419803,4b69873f-5e41-4c9d-87db-2fe9b64fcc78,0,122,2021-08-06T13:21:39.727+0000,true,true,127,Submitted titles don’t match current ISWC Documentation,CAR,T9804132034,LY1kfY71Ckc,308,INT TEST tmCxjURZbmbmeKA iADswkWZjHBvbGQ BxAUdwMHBdVHLPt x,Agency,BAKER BILL ;REEVES KEN,1856409;1906032,0,,false,,2021-08-06,13,300,122,AKKA-LAA,LATVIA,2008-03-04 10:27:40.0000000,2008-03-04 10:27:40.0000000,7,1,300,WIDCENTRE
596c4e10-d695-408f-9d4d-eab2cf9d7f34,64bcc1ac-9e27-4bf4-87d0-efd4d7980fa5,0,122,2021-08-06T13:45:47.133+0000,true,true,127,Submitted titles don’t match current ISWC Documentation,CAR,T9804135691,W3zUjYGiWykB2LA5,308,INT TEST QwfMKjbEBSOyVDB fEXOtelJzrtRPMO EdHhjhCrEvHKVzU x,Agency,BAKER BILL ;REEVES KEN,1856409;1906032,0,,false,,2021-08-06,13,300,122,AKKA-LAA,LATVIA,2008-03-04 10:27:40.0000000,2008-03-04 10:27:40.0000000,7,1,300,WIDCENTRE
a2191074-7c99-45e3-8dd0-cbfabbc7fb82,fd149335-26d9-45d6-9e60-3aef02f8e818,0,122,2021-08-06T14:04:44.049+0000,true,true,127,Submitted titles don’t match current ISWC Documentation,CAR,T9804138725,b78bEY5rQRDseZvaY,308,INT TEST oryviMOnvDXNgnY TJKKeRMOnutJpMq xWQWGjYWKjvoKke x,Agency,BAKER BILL ;REEVES KEN,1856409;1906032,0,,false,,2021-08-06,14,300,122,AKKA-LAA,LATVIA,2008-03-04 10:27:40.0000000,2008-03-04 10:27:40.0000000,7,1,300,WIDCENTRE
e4d22877-8ad8-46ba-a922-1ab6b5418eda,1e41dd99-ab3f-447f-9530-dd15d70f561a,0,122,2021-08-06T16:05:24.827+0000,true,true,127,Submitted titles don’t match current ISWC Documentation,CAR,T9804141444,zfrieL9ej3rWh,308,INT TEST NVJHAOZOuWJILyq HiDqIhwvaznQEAt TsSRCfwFHYGnxUf x,Agency,BAKER BILL ;REEVES KEN,1856409;1906032,0,,false,,2021-08-06,16,300,122,AKKA-LAA,LATVIA,2008-03-04 10:27:40.0000000,2008-03-04 10:27:40.0000000,7,1,300,WIDCENTRE
55a321fb-b45a-4725-962a-b915d4b4c56f,be30d35c-3435-4326-a979-f95b7fbb2694,0,122,2021-08-05T13:36:13.800+0000,true,true,127,Submitted titles don’t match current ISWC Documentation,CAR,T9804108358,AmeSSdSWJhjiV5Qz,308,INT TEST vHQtApxmqmXVRFZ xfgpDhZEQoOOGMZ eKTFJsQcbpCQESU x,Agency,BAKER BILL ;REEVES KEN,1856409;1906032,0,,false,,2021-08-05,13,300,122,AKKA-LAA,LATVIA,2008-03-04 10:27:40.0000000,2008-03-04 10:27:40.0000000,7,1,300,WIDCENTRE
5f7c61de-8a1b-4537-9d3f-53c6fdba7f90,85cffb91-3a58-4d1f-ab35-f2409b597d80,0,122,2021-08-05T14:46:37.105+0000,true,true,127,Submitted titles don’t match current ISWC Documentation,CAR,T9804111828,eu5TpuoMKz,308,INT TEST CPhLIEvDVMjzPdI PlOEXfOXDcPrhML enitYvCISzgJFLw x,Agency,BAKER BILL ;REEVES KEN,1856409;1906032,0,,false,,2021-08-05,14,300,122,AKKA-LAA,LATVIA,2008-03-04 10:27:40.0000000,2008-03-04 10:27:40.0000000,7,1,300,WIDCENTRE
8c8e0b86-a822-4aba-93b0-ad0f34

In [0]:
# cosmos service set up
from ediparser.parser.services.key_vault_configuration import \
    KeyVaultConfiguration
from ediparser.parser.services.cosmos_service import CosmosService
import json
from pyspark.sql.functions import *

database = 'ISWC'
container = 'AgentRuns'
container_link = "dbs/{}/colls/{}".format(database, container)

config = KeyVaultConfiguration(dbutils)

cosmos_connection_string = config.get_secret('AzureKeyVaultSecret-ISWC-ConnectionString-ISWCCosmosDb', True)

cosmos_service = CosmosService(cosmos_connection_string, database, container)

runs = []
if filterAgency != 'All':
  runs = cosmos_service.get_agent_runs_by_agency(filterAgency, fromDate, toDate)

def format_date_string(date_to_format):
  if not date_to_format.endswith('Z'):
    date_to_format += 'Z'
    
  d = datetime.strptime(date_to_format, '%Y-%m-%dT%H:%M:%S.%fz')
  formatted_date = d.strftime("%m/%d/%Y %H:%M:%S")
  return  formatted_date

for run in runs:
  run['RunStartDate'] = format_date_string(run['RunStartDate'][:26])
  
  
  if run.get('RunEndDate') is not None:
    run['RunEndDate'] = format_date_string(run['RunEndDate'][:26])

In [0]:
cols = ['Run ID', 'Start Date', 'End Date', 'Run Completed', 'Sent', 'Successful', 'Updates', 'New Records', 'Business Rule Rejected', 'Technical Rejection', 'Agent Version']

if len(runs) > 0:
  runs_df = spark.createDataFrame(runs)

  mapping = dict(zip(['id', 'RunStartDate', 'RunEndDate', 'RunCompleted', 'OverallSent', 'SuccessfulCount', 'UpdateRecordCount', 'NewRecordCount', 'BusinessRejectionCount', 'TechnicalRejectionCount',  'AgentVersion' ], cols))

  runs_to_display = runs_df.select([col(c).alias(mapping.get(c, c)) for c in runs_df.columns])
  
  display(runs_to_display.select(cols))